## LDAvis 를 이용한 k-means 결과 시각화

각 군집의 키워드 점수를 계산한 뒤, 이를 LDAvis 의 키워드 점수에 이용할 수도 있지만, day4 에서 NMF 를 LDAvis 로 시각화 한 것 처럼 군집화의 결과를 토픽 모델링의 결과로 변환하여 LDAvis 에 입력할 수도 있습니다.

Bag of words model, x 를 로딩합니다. `x` 는 (n docs, n terms) 입니다. 군집화를 위하여 TF-IDF 변환을 할 수도 있습니다.

In [1]:
import sys
import config
from navernews_10days import get_bow
from sklearn.feature_extraction.text import TfidfTransformer
sys.path.append('./clustering4docs/')

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')
print(x.shape)

x_tfidf = TfidfTransformer().fit_transform(x)

soynlp=0.0.492
added lovit_textmining_dataset
(30091, 9774)


Spherical k-means 를 이용하여 군집화를 수행하였습니다. `labels` 는 (n docs, ) 크기의 numpy.ndarray 입니다.

In [2]:
from soyclustering import SphericalKMeans

kmeans = SphericalKMeans(
    n_clusters = 200,
    init = 'similar_cut',
    verbose = True,
    random_state = 0,
)

labels = kmeans.fit_predict(x_tfidf)

initialization_time=0.264676 sec, sparsity=0.00727
n_iter=1, changed=29962, inertia=19568.605, iter_time=1.530 sec, sparsity=0.22
n_iter=2, changed=6433, inertia=15441.697, iter_time=1.558 sec, sparsity=0.194
n_iter=3, changed=2557, inertia=14879.910, iter_time=1.557 sec, sparsity=0.187
n_iter=4, changed=1339, inertia=14694.164, iter_time=1.525 sec, sparsity=0.184
n_iter=5, changed=814, inertia=14611.636, iter_time=1.535 sec, sparsity=0.182
n_iter=6, changed=503, inertia=14564.093, iter_time=1.536 sec, sparsity=0.181
n_iter=7, changed=278, inertia=14536.708, iter_time=1.560 sec, sparsity=0.181
n_iter=8, changed=189, inertia=14525.633, iter_time=1.578 sec, sparsity=0.181
n_iter=9, changed=144, inertia=14516.308, iter_time=1.563 sec, sparsity=0.181
n_iter=10, changed=129, inertia=14506.995, iter_time=1.578 sec, sparsity=0.181


문서 군집화는 "하나의 문서는 하나의 토픽으로 이뤄져있다"고 가정한 것과 같습니다. 예를 들어 3 번 군집에 해당하는 문서라면 이 문서의 토픽 벡터는 (0, 0, 0, 1, 0, ...) 로 생각하면 됩니다.

`kmeans_to_topic_model` 함수는 군집화 결과인 `labels` 와 bag of words model 인 `x` 를 입력받습니다. `x` 에서 각 군집에 해당하는 문서들을 선택한 뒤, row sum 을 하면 군집 내의 단어 빈도수 벡터가 만들어집니다. 만약 이 값이 0 이라면 empty vector 로, 그렇지 않다면 총 합으로 이를 나누어 L1 norm 이 1 이 되도록 만듭니다.

아래처럼 if 문을 이용한 이유는 freq_sum 이 0 일 경우, row 가 infinite 가 될 수 있기 때문입니다. 대신 empty row 는 `probablity_normalization` 함수를 이용하여 각 element 의 값이 군집 개수의 역수 (1/k) 가 되도록 만듭니다.

```python
for c in range(n_clusters):
    if freq_sum == 0:
        continue
    topic_term_prob[c] = topic_term_freq / freq_sum
```

In [3]:
import numpy as np

def kmeans_to_topic_model(labels, x):
    n_clusters = np.unique(labels).shape[0]
    n_docs, n_terms = x.shape

    doc_topic_prob = np.zeros((n_docs, n_clusters))
    topic_term_prob = np.zeros((n_clusters, n_terms))

    for c in range(n_clusters):
        idx = np.where(labels == c)[0]
        doc_topic_prob[idx, c] = 1
        topic_term_freq = x[idx].sum(axis=0)
        freq_sum = topic_term_freq.sum()
        if freq_sum == 0:
            continue
        topic_term_prob[c] = topic_term_freq / freq_sum

    return topic_term_prob, doc_topic_prob

def sum_bow(bow):
    doc_lengths = np.asarray(bow.sum(axis=1)).reshape(-1)
    term_frequency = np.asarray(bow.sum(axis=0)).reshape(-1)
    return doc_lengths, term_frequency

def probablity_normalization(prob):
    row_sum = prob.sum(axis=1)
    base = 1 / prob.shape[1]
    prob[np.where(row_sum == 0)[0]] = base
    return prob

`kmeans_to_topic_model` 함수를 이용하여 문서의 토픽 벡터와 토픽의 단어 벡터를 만듭니다.

Bag of words model 의 row sum 을 계산하면 데이터 전체에서의 단어 빈도수를 계산할 수 있고, column sum 을 계산하면 문서에 등장한 단어 개수 (문서 길이)를 계산할 수 있습니다. `sum_bow` 함수를 이용하여 이를 계산합니다.

In [4]:
topic_term_prob, doc_topic_prob = kmeans_to_topic_model(labels, x)
topic_term_prob = probablity_normalization(topic_term_prob)
doc_topic_prob = probablity_normalization(doc_topic_prob)

doc_lengths, term_frequency = sum_bow(x)

LDAvis 는 다섯개의 정보를 반드시 입력해야 합니다. 우리는 앞선 과정에서 이를 모두 준비하였습니다.

- `topic_term_prob` : numpy.ndarray 형식의 (토픽, 단어) 확률 행렬
- `doc_topic_prob`  : numpy.ndarray 형식의 (문서, 토픽) 확률 행렬
- `doc_lengths`     : numpy.ndarray 형식의 문서의 단어 개수
- `idx_to_vocab`    : list of str 형식의 단어 인덱스
- `term_frequency`  : numpy.ndarray 형식의 단어 빈도 벡터

Argument `R` 은 LDAvis 의 오른쪽 화면에 출력되는 키워드의 개수입니다. `plot_opts` 는 x, y 축의 이름입니다. `mds` 를 `tsne` 입력하면 t-SNE 를 이용하여 centroids 의 2차원 표현을 학습합니다. 기본값은 PCA 를 이용한 2차원 표현의 학습입니다.

prepared data 의 크기가 클 경우에는 IPython Notebook 의 결과값에 그림이 출력되지 않기도 합니다. `save_html` 을 이용하여 HTML 파일로 저장한 뒤 이를 확인합니다.

In [5]:
from pyLDAvis import prepare, show, save_html

prepared_data = prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    idx_to_vocab,
    term_frequency,
    mds = 'tsne',
    plot_opts = {'xlab': 't-SNE1', 'ylab': 't-SNE2'},
    R = 30 # num of displayed terms
)

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [6]:
save_html(prepared_data, 'kmeans_to_ldavis.html')